conda activate getcontact

There are several scripts:
1. Naive alignment: simply calculate the frequency by shifting the whole "short" sequence by adding tap.
2. Global alignment: sequence without gap alignment, sequence with gap alignment

Remember that you could always use BLAST to do local alignment (find similar local region, instead of aligning the whole sequence), which use a more complex alignment algorithm.

When to use local alignment and global alignemnt?
1. When the two sequences you want to compare is similar in length -> Global alignment.
2. When the two sequences have very different length, or if more than one domain of seq1 appears in seq2 -> Local alignment. Remember the repeat region of seq1 can be confusing for global alignment.


In [ ]:
#conda activate getcontact

#If the sequences are the same length
#First transpose the row and column
sequences = ["GCFDDSQ",
             "MCEEDFQ",
             "GCDEDSF",]
transposed = [list(row) for row in zip(*sequences)]
print(transposed)

#Use Counter to count the frequency for each column
from collections import Counter
consensus_sequence = "".join([max(Counter(data), key=Counter(data).get) for data in transposed])
print(consensus_sequence)

#Another way of getting the most common key and value are
#consensus_sequence = "".join([Counter(data).most_common(1)[0][0] for data in transposed])
#print(consensus_sequence)

#The consesus sequence above is list comprehension for the following code
#consensus_sequence = []
#for data in transposed:
#    freq = Counter(data)
#    max_value = max(freq, key=freq.get)
#    consensus_sequence.append(max_value)
#consensus_sequence = "".join(consensus_sequence)

In [60]:
#If the two sequences are not the same length
#The idea is to first rank based on length
#Then add different number of "-" for the short one, and compare which global alignment can have the max repeat
from collections import Counter
sequences = ["MCEEDFSKQFT",
             "GCDDDSFT",]
#Rank based on length, shortest first
ranked_sequences = sorted(sequences, key=len, reverse=False)

#Generate all padded versions of the shortest sequence
length_diff = len(ranked_sequences[1]) - len(ranked_sequences[0])
padded_sequences = ["-" * n + ranked_sequences[0] for n in range(length_diff + 1)]

#Transpose with different distance gap
rank_alignment = {}
for padded in padded_sequences:
    # Align the padded sequence with the longer sequence
    transposed = zip(padded, ranked_sequences[1])
    
    #Find consensus by counting maximum repeat
    consensus = []
    max_sum = 0
    for column in transposed:
        most_common = Counter(column).most_common(1)[0]  # Get the most common element and its count, output in tuples
        #print("Most common:", most_common)
        consensus.append(most_common[0])
        max_sum += most_common[1]
        
    # Store the consensus and its score
    rank_alignment["".join(consensus)] = max_sum
    
# Find the best alignment
best_alignment = max(rank_alignment, key=rank_alignment.get)

best_alignment_matrix = [best_alignment, ranked_sequences[1]]
# Output results
print("Padded sequences:", padded_sequences)
print("Consensus alignments with scores:", rank_alignment)
print("Best alignment:", best_alignment)
print(best_alignment_matrix)

Padded sequences: ['GCDDDSFT', '-GCDDDSFT', '--GCDDDSFT', '---GCDDDSFT']
Consensus alignments with scores: {'GCDDDSFT': 10, '-GCDDDSFT': 11, '--GCDDDSFT': 11, '---GCDDDSFT': 13}
Best alignment: ---GCDDDSFT
['---GCDDDSFT', 'MCEEDFSKQFT']


In [13]:
#zip function usually only take the shortest sequence to zip, but we could also choose to zip the longest sequence
#Future more, since we only compare two sequences, we dont really need to count the frequency, but just compare if the two base is the same
from itertools import zip_longest
sequences = ["MCEEDFSKQFT", "DFSK"]
# Sort sequences by length
sequences = sorted(sequences, key=len, reverse=False)
max_score = 0
best_alignment = None

# Generate all possible paddings for seq1
length_diff = len(sequences[1]) - len(sequences[0])
padded_sequences = ["-" * n + sequences[0] for n in range(length_diff + 1)]

# Test all alignments
for padded in padded_sequences:
    transposed = zip_longest(padded, sequences[1], fillvalue='-')
    score = sum(1 for a, b in transposed if a == b)
    if score > max_score:
        max_score = score
        best_alignment = padded
print(best_alignment, max_score)

----DFSK 4


In [29]:
#The idea is to align all the sequences to the longest sequence
sequences = ["MCEEDFSKQFT", "CEEDF", "DFSK"]
sequences = sorted(sequences, key=len, reverse=True)
consensus = sequences[0]
aligned_sequence = []

def align(long, short):
    max_score = 0
    best_alignment = None

    # Generate all possible paddings for seq1
    length_diff = len(long) - len(short)
    padded_sequences = ["-" * n + short for n in range(length_diff + 1)]

    # Test all alignments
    for padded in padded_sequences:
        transposed = zip_longest(padded, long, fillvalue='-')
        score = sum(1 for a, b in transposed if a == b)
        if score > max_score:
            max_score = score
            best_alignment = padded
    return best_alignment, max_score

for a in range(len(sequences)): 
    best_align, score = align(consensus,sequences[a])
    aligned_sequence.append(best_align)

for element in aligned_sequence:
    print(element)


MCEEDFSKQFT
-CEEDF
----DFSK


In [51]:
#Pevious we were comparing whether two base are exactly the same and align base on that
#Now we want to align based on similarity
#For example, D and E are similar, K and R are similar
#So we can build a similar matrix and find the alignment
sequences = ["MCEEDFSKQFT", "CDEDWSH", "DWTR"]
sequences = sorted(sequences, key=len, reverse=True)
consensus = sequences[0]
aligned_sequence = {}

def align(long, short):
    similarity_matrix = {}

    # Similar residues based on charge, polarity, and side-chain properties
    similarity_matrix.update({
        ("D", "E"): 4, ("K", "R"): 4, ("N", "Q"): 4,  # Similar polar/charged residues
        ("S", "T"): 4, ("F", "Y"): 4,                # Similar polar uncharged residues
        ("I", "L"): 4, ("L", "V"): 4, ("I", "V"): 4,  # Hydrophobic residues
        ("A", "G"): 3,                               # Small residues
        ("W", "Y"): 3, ("W", "F"): 3,                # Aromatic residues
    })

    max_score = 0
    best_alignment = None
    # Generate all possible paddings for seq1
    length_diff = len(long) - len(short)
    padded_sequences = ["-" * n + short for n in range(length_diff + 1)]

    # Test all alignments
    for padded in padded_sequences:
        transposed = zip_longest(padded, long, fillvalue='-')
        sum = 0
        for a, b in transposed:
            if a == b:
                sum +=5
            elif (a, b) in similarity_matrix:
                sum = sum + similarity_matrix[(a,b)]
            elif (b,a) in similarity_matrix:
                sum = sum + similarity_matrix[(b,a)]
            else:
                sum = sum
        if sum > max_score:
            max_score = sum
            best_alignment = padded
        
    return best_alignment, max_score

for a in range(len(sequences)): 
    best_align, score = align(consensus,sequences[a])
    aligned_sequence.update({best_align:score})
    
for key in aligned_sequence:
    print(key, aligned_sequence[key])

MCEEDFSKQFT 55
-CDEDWSH 27
----DWTR 16


# Global alignment
1. Gap penalty should be larger than mismatch penalty: Because insertion / deletion is less common than mutation. There are two types of gap penalty: linear gap penalty and affine gap penalty. The affine gap penalty decrease the panalty for more 'accumulated' gap.
2. 

In [117]:
# If you want to do global alignment with gap in middle of the sequence
# Use linear gap penalty
import numpy as np # type: ignore

# Define scoring rules
gap_penalty = -2
match_score = 5
similarity_matrix = {
    ("D", "E"): 4, ("K", "R"): 4, ("N", "Q"): 4,
    ("S", "T"): 4, ("F", "Y"): 4,
    ("I", "L"): 4, ("L", "V"): 4, ("I", "V"): 4,
    ("A", "G"): 3, ("W", "Y"): 3, ("W", "F"): 3,
}

def calculate_score(a, b):
    """Calculate the score between two residues."""
    if a == b:
        return match_score
    elif (a, b) in similarity_matrix or (b, a) in similarity_matrix:
        return similarity_matrix.get((a, b), similarity_matrix.get((b, a), 0))
    elif a == "-" or b == "-":
        return gap_penalty
    else:
        return 0

def global_align(seq1, seq2):
    """Perform global alignment between two sequences."""
    n = len(seq1)
    m = len(seq2)

    # Initialize scoring matrix
    dp = np.zeros((n + 1, m + 1))
    for i in range(1, n + 1):
        dp[i][0] = i * gap_penalty
    for j in range(1, m + 1):
        dp[0][j] = j * gap_penalty

    # Fill scoring matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = dp[i - 1][j - 1] + calculate_score(seq1[i - 1], seq2[j - 1])
            delete = dp[i - 1][j]
            insert = dp[i][j - 1]
            maximum = max(match, delete, insert)
            if maximum == match:
                dp[i][j] = match
            elif maximum == delete:
                dp[i][j] = delete - 2
            else:
                dp[i][j] = insert - 2
            #dp[i][j] = max(match, delete, insert)

    print(dp)
    # Traceback to get alignment
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + calculate_score(seq1[i - 1], seq2[j - 1]):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            dp[i][j] = 99.0
            i -= 1
            j -= 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + gap_penalty:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append("-")
            dp[i][j] = 88.0
            i -= 1
        else:
            aligned_seq1.append("-")
            aligned_seq2.append(seq2[j - 1])
            dp[i][j] = 77.0
            j -= 1

    # Reverse to get the correct order
    print(dp)
    return "".join(reversed(aligned_seq1)), "".join(reversed(aligned_seq2)), dp[n][m]


# Input sequences
sequence_dic = {'3ult.pdb': 'PNTSSFWWRSGSN',
                'aest': 'NTYRSGS',
                #'test': 'PNTISGSNNTVRSG'
                }
sequences = list(sequence_dic.values())

#If you want to reorder based on names in dic
sorted_sequence_dic = {key: sequence_dic[key] for key in sorted(sequence_dic)}
sequences = list(sorted_sequence_dic.values())

#If you want to reorder the sequence based on length
#sequences = sorted(sequences, key=len, reverse=True)

aligned_results = []

# Perform global alignment against the first sequence
for seq in sequences[1:]:
    aligned_seq1, aligned_seq2, alignment_score = global_align(sequences[0], seq)
    aligned_results.append((aligned_seq1, aligned_seq2, alignment_score))

# Print results
for aligned_seq1, aligned_seq2, alignment_score in aligned_results:
    align_conclude = ""
    transposed = zip(aligned_seq1,aligned_seq2)
    for a, b in transposed:
        if a == b:
            align_conclude += "*"
        elif (a, b) in similarity_matrix:
            align_conclude += "s"
        elif (b, a) in similarity_matrix:
            align_conclude += "s"
        else:
            align_conclude += "_"
    #Add the name of sequence to output result
    key_seq1 = next((k for k, v in sequence_dic.items() if v == aligned_seq1.replace("-","")), None)
    key_seq2 = next((k for k, v in sequence_dic.items() if v == aligned_seq2.replace("-","")), None)
    #right adjust and padded with space
    print(f"{key_seq1[0:5].rjust(7)} {aligned_seq1}")
    print(f"{key_seq2[0:5].rjust(7)} {aligned_seq2}")
    print("align_conclude"[0:5].rjust(7) + f" {align_conclude} {alignment_score}")
    print("\n")


[[  0.  -2.  -4.  -6.  -8. -10. -12. -14.]
 [ -2.   0.  -2.  -4.  -6.  -8. -10. -12.]
 [ -4.   3.   1.  -1.  -3.  -5.  -7.  -9.]
 [ -6.   1.   8.   6.   4.   2.   0.  -2.]
 [ -8.  -1.   6.   8.   6.   9.   7.   5.]
 [-10.  -3.   4.   6.   8.  11.   9.  12.]
 [-12.  -5.   2.   8.   6.   9.  11.  10.]
 [-14.  -7.   0.   6.   8.   7.   9.  11.]
 [-16.  -9.  -2.   4.   6.   8.   7.   9.]
 [-18. -11.  -4.   2.   9.   7.   8.   7.]
 [-20. -13.  -6.   0.   7.  14.  12.  13.]
 [-22. -15.  -8.  -2.   5.  12.  19.  17.]
 [-24. -17. -10.  -4.   3.  10.  17.  24.]
 [-26. -19. -12.  -6.   1.   8.  15.  22.]]
[[  0.  -2.  -4.  -6.  -8. -10. -12. -14.]
 [ 88.   0.  -2.  -4.  -6.  -8. -10. -12.]
 [ -4.  99.   1.  -1.  -3.  -5.  -7.  -9.]
 [ -6.   1.  99.   6.   4.   2.   0.  -2.]
 [ -8.  -1.  88.   8.   6.   9.   7.   5.]
 [-10.  -3.  88.   6.   8.  11.   9.  12.]
 [-12.  -5.   2.  99.   6.   9.  11.  10.]
 [-14.  -7.   0.  88.   8.   7.   9.  11.]
 [-16.  -9.  -2.  88.   6.   8.   7.   9.]
 [-18. -11